In [1]:
import argparse
import os
from utils import *
import re
import json
import subprocess
from pathlib import Path


import importlib

# importlib.reload(utils)

DASHSCOPE_API_KEY = "sk-a15dd69d27524486a759b7ea0e706a87"


# 0_preprocess.py

In [20]:
# save the image pairs to a json file

input_latex_path = "/Users/v1p0r/Documents/DDL_Fall25/term_project/protopy_agent/data/input/latex/arXiv-2510.14980v1"
output_path = "/Users/v1p0r/Documents/DDL_Fall25/term_project/protopy_agent/data/output"
paper_name = os.path.basename(input_latex_path)

pattern = r"\\includegraphics(?:\[.*?\])?\{(.*?)\}"

image_pairs = {}

# read the latex fileso
for index, latexfile in enumerate(extract_raw_figures(input_latex_path)):
    
    image_path_line = [line for line in latexfile['snippet'].splitlines() if "\\includegraphics" in line][0]
    
    filename = re.findall(pattern, image_path_line)

    image_pairs[index] = {"image": filename[-1], "caption": latexfile['snippet']}
    # print(type(i))
    # caption_line = [
    #     line for line in i['snippet'].splitlines()
    #     if re.search(r'\\caption(?:\s*\[[^\]]*\])?\s*\{', line)
    # ][0]

    # print(type(caption_line))
    # papers[file].append(( \
    #     os.path.join(input_path,file, extract_figure_name_and_caption(image_path_line, caption_line)[0]), \
    #     extract_figure_name_and_caption(image_path_line, caption_line)[1]
    # ))

full_path = os.path.join(output_path, paper_name)
os.makedirs(full_path, exist_ok=True)

with open(full_path + '/' + 'image_pairs.json', 'w',encoding="utf-8") as f:
    json.dump(image_pairs, f)

In [3]:
print(len(image_pairs))

31


# 1_image_agent.py
## a. filter the image pairs based on the caption

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from openai import OpenAI
import json
model_name = "qwen3-vl-flash"

/Users/v1p0r/miniconda3/envs/vllm_agent/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [5]:
# load the image pairs from preprocess.py
with open(full_path + '/' + 'image_pairs.json', "r", encoding="utf-8") as f:
    image_pairs = json.load(f)

payload = json.dumps(image_pairs, ensure_ascii=False, separators=(",", ":"))

In [6]:
# initialize the OpenAI competiable client for qwen3-vl-flash
client = OpenAI(
    api_key=DASHSCOPE_API_KEY,
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1",
)

def api_call(model_name, prompt):
    completion = client.chat.completions.create(
            model=model_name, 
            messages=prompt
    )
    return completion

In [7]:
filtering_prompt = [{'role': 'system', 'content': """
You are a strict figure classifier for a research paper.

You receive a JSON object mapping string IDs to:
{
    "0": {
        "image": "path or filename",
        "caption": "LaTeX figure environment including caption"
    },
    "1": {
        "image": "path or filename",
        "caption": "LaTeX figure environment including caption"
    },
    
}

Your job is to select ONLY figures that are:
- Identify only figures that depict **model architectures**, **pipelines**, or **system workflows**.
- Exclude all others, including metric/performance plots, ablations, task illustrations, examples, galleries, datasets, failure cases, and editor screenshots.
- For each selected figure, extract the **human-readable caption text** (remove LaTeX commands like \\begin, \\end, \\vspace, \\label, \\includegraphics, etc.).
- Return that caption in plain, readable English — no LaTeX, no comments.

You MUST EXCLUDE figures that are primarily:
- Metrics, curves, score plots, RL training curves
- Performance comparisons, ablations, Best@N, validity rates, reward curves
- Qualitative examples, galleries of results, synthesized samples
- Task/environment illustrations, dataset examples, failure cases, case studies
- Editor UI screenshots, block catalogs, low-level component lists without a full system pipeline

Use ONLY the provided `caption` text (and, if helpful, filename hints like 'RL_Metrics').

Do not include explanations, comments, or extra fields.
"""},


{'role': 'user', 'content': f"""

Below is a JSON object containing figure data from a research paper.

{payload}


Output format:
Return **strictly valid JSON** of this exact format:
```json
{{
    
    "id": "clear caption text",
    "4": "clear caption text",
    "8": "clear caption text"
    
  
}}
```


"""}
]

response = api_call(model_name, filtering_prompt)


In [8]:
response_json = json.loads(response.model_dump_json())

# print(json.dumps(response_json, indent=2, ensure_ascii=False))


In [9]:
output_json = response_json["choices"][0]["message"]["content"]
filtered_image_pairs = json.loads(output_json.replace("```json", "").replace("```", "").strip())

print(filtered_image_pairs)


{'5': 'Our agentic machine design workflow.', '2': 'Demonstration of the default XML representation and our construction tree representation. Parent block info is in blue and child info is in red.'}


In [10]:

# print(Path(os.path.join(output_path, paper_name,os.path.basename(filename))).with_suffix('.png'))

## b. extract the information from the image

In [11]:
# image to png to b64

from pathlib import Path

for key, caption in filtered_image_pairs.items():
    filtered_image_pairs[key] = {}

    filename = os.path.join(input_latex_path, image_pairs[key]["image"])
    if filename.lower().endswith('.pdf'):
        png_output_path = Path(os.path.join(output_path, paper_name,os.path.basename(filename))).with_suffix('.png')
        subprocess.run(['bash','/Users/v1p0r/Documents/DDL_Fall25/term_project/protopy_agent/pdf2png.sh', filename, png_output_path])

    filtered_image_pairs[key] = {"image": image_pairs[key]["image"], \
        "caption": caption, \
        "b64": image_to_base64(png_output_path)
        }




✅ Saved: /Users/v1p0r/Documents/DDL_Fall25/term_project/protopy_agent/data/output/arXiv-2510.14980v1/besiegefield_pipeline_compressed.png
✅ Saved: /Users/v1p0r/Documents/DDL_Fall25/term_project/protopy_agent/data/output/arXiv-2510.14980v1/xml_intro_v2_cropped.png


In [18]:
image_list = []
for key, image_pair in filtered_image_pairs.items():
    caption = image_pair["caption"]
    image_b64 = image_pair["b64"]
    extraction_prompt = [{'role': 'system', 'content': '''
    You are an expert vision-language model specialized in understanding figures from computer science and machine learning papers.

    Given:
    1. A caption describing a figure.
    2. The figure image itself.

    Your tasks:

    1. Classify the figure into exactly ONE of the following categories:
    - "model_architecture": diagrams that show layers, blocks, components of a model or network.
    - "pipeline": diagrams that show a sequence of processing steps or modules for data or experiments (ingest → preprocess → train → eval, etc.).
    - "system_workflow": diagrams that show interactions between multiple components, agents, services, or tools in a larger system (e.g., multi-agent system, platform, or end-to-end toolchain).
    - "other": if none of the above clearly apply.

    2. Extract structured information from the figure that would help RECONSTRUCT the model, pipeline, or system in code.
    Be as concrete and implementation-oriented as possible:
    - List key components/blocks/modules with concise names and roles.
    - List edges / data flows (source → target, what is passed).
    - List stages or steps in order, if applicable.
    - Include inputs and outputs (data types, files, models, agents, APIs, etc.).
    - Include any loops, conditions, or control flow if visible.
    - Prefer information grounded in the VISUAL content; use the caption only as supporting context.
    - Do NOT invent components that are not clearly implied by the figure.

    3. Output MUST be valid JSON only. No extra text.

    JSON schema (strict):

    {
    "category": "model_architecture" # "pipeline" or "system_workflow" or "other",
    "extracted": {
        "components": [
        {
            "name": "string",
            "type": "string",
            "role": "string"
        }
        ],
        "flows": [
        {
            "source": "string",
            "target": "string",
            "data": "string"
        }
        ],
        "stages": [
        {
            "name": "string",
            "description": "string"
        }
        ],
        "inputs": ["string"],
        "outputs": ["string"]
    }
    }
    '''},{
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Analyze the following figure and respond in the required JSON schema."
            },
            {
                "type": "text",
                "text": f"Caption: {caption}"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{image_b64}"
                }
            }
        ]
    }]
    response = api_call(model_name, extraction_prompt)
    output_json = json.loads(response.model_dump_json())
    content = output_json["choices"][0]["message"]["content"]
    image_list.append(content)



In [ ]:
with open(f'{output_path}/{paper_name}/image_extracted_information.json', 'w') as f:
        json.dump(content, f)

In [13]:
# # response.model_dump_json()
# x =json.loads(response.model_dump_json())
# # response = api_call(model_name, extraction_prompt)
# # parse:
# content = response.model_dump_json()["choices"][0]["message"]["content"]

In [14]:
# print(x["choices"][0]["message"]["content"])

In [15]:
# print(output_path)

In [16]:
# x = json.loads(content)["choices"][0]["message"]["content"]
# with open(f'{output_path}/image_extracted_information.json', 'w') as f:
#     json.dump(x, f)




